In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import math

### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 

#### Questions:

a) Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

b) Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 

Please perform all the necessary computations "by hands" rather than using python code.

In [4]:
#P(Stolen|{Red,SUV,Domestic}) = 
3/5.0 * 1/5.0 * 2/5.0 * 0.5

0.024

In [5]:
#P(Not Stolen|{Red, SUV,Domestic}) = 
2/5.0 * 3/5.0 * 3/5.0 * 0.5

0.07200000000000002

#Classification 1. P(Stolen/Red)  = P(Red/Stolen)*P(Stolen)/P(Red)
(3/5.0)*(0.5)*0.5
#Classification 1. P(Not Stolen/Red)  = P(Red/Not Stolen)*P(Not Stolen)/P(Red)
(2/5.0)*0.5*0.5
#Classification 1. P(Stolen/SUV)  = P(SUV/Stolen)*P(Stolen)/P(SUV)
(1/5.0)*(0.5)*(0.4)
#Classification 1. P(Not Stolen/Red)  = P(SUV/Not Stolen)*P(Not Stolen)/P(SUV)
(3/5.0)*(0.5)(0.4)
#Classification 1. P(Stolen/Red)  = P(Domestic/Stolen)*P(Stolen)/P(Domestic)
(2/5.0)*(0.5)/(0.5)
#Classification 1. P(Not Stolen/Red)  = P(Domestic/Not Stolen)*P(Not Stolen)/P(Domestic)
(3/5.0)*(0.5)/(0.5)

In [6]:

y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

In [7]:
data

,Stolen?,Color,Type,Origin
0,1,1,1,1
1,0,1,1,1
2,1,1,1,1
3,0,2,1,1
4,1,2,1,2
5,0,2,2,2
6,1,2,2,2
7,0,2,2,1
8,0,1,2,2
9,1,1,1,2


In [8]:
def trainNaiveBayesDiscrete(trainData):
      #training discrete Naive Bayes Classifier
    tY=trainData.loc[:,trainData.columns[0]]
    m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
      #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
    dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
      #split the training data between two labels
    ind1=tY==0
    ind2=tY==1
    #estimate P(y=b)  
    dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
    dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
      #estimate conditional probabilities P(x|y=b)
    for j in trainData.columns[1:]:
        for i in range(1,m+1):
            dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
            
            dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
            
    return dp

In [9]:
def classifyNaiveBayesDiscrete(classData,dp):
    #classifying using trained discrete Naive Bayes Classifier
    Y=classData[classData.columns[0]]*0 #initialize the empty array 
    for i in classData.index: #for al records to classify
    #start with the priors
        P1=dp[0][classData.columns[0]][1]; 
        P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
        for j in classData.columns[1:]:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
    return Y

### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:

a)Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

b)Perform the classification for the test sample. 

c)Compare your result to y_test and report the classification accuracy.

In [11]:
def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

In [12]:
def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

In [3]:
data_train=pd.read_csv("dia_train.csv",index_col=0) 
y_train=data_train.iloc[:,1] 
X_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("dia_test.csv",index_col=0)
y_test=data_test.iloc[:,1]
X_test=data_test.iloc[:,2:]

In [14]:
dp = trainNaiveBayes(data_train)

In [15]:
dp

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
mu1,0.699153,2.69091,111.467,69.2061,27.2,127.006,31.7091,0.468685,28.3939
sigma1,NaN,2.61006,24.6919,11.7133,10.4369,91.4861,6.33761,0.29175,8.53736
mu2,0.300847,4.07042,144.141,74.5634,33.4789,209.211,35.2239,0.639042,35.7887
sigma2,NaN,3.51862,30.6265,13.7993,9.7627,126.921,6.25849,0.439042,10.2635


In [16]:
C=classifyNaiveBayes(data_test,dp) #classify test set
#classification accuracy (over test set)
100.0*sum(C==data_test.y)/len(C)

79.11392405063292

In [17]:
#also report accuracy of the classifier over the training - it is slightly higher, although not that higher
Ct=classifyNaiveBayes(data_train,dp)
100.0*sum(Ct==data_train.y)/len(Ct)

76.27118644067797

### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.

#### Questions:

a) Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

b) Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook). 

In [18]:
data_train=pd.read_csv("EM_train.csv",index_col=0)
y_Label_train=data_train.iloc[:,1] 
X_Label_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("EM_test.csv",index_col=0)
y_Label_test=data_test.iloc[:,1]
X_Label_test=data_test.iloc[:,2:]

data_Unlabel=pd.read_csv("EM_Unlabel.csv",index_col=0)
X_Unlabel=data_Unlabel.iloc[:,1:]

In [19]:
data_train

,y,0,1,2
3,0,4,6,6
135,1,2,2,2
25,0,2,6,2
111,0,6,6,6
76,1,1,2,2
24,0,3,4,4
89,0,6,1,6
22,1,1,3,1
34,1,5,1,6


In [20]:
dp=trainNaiveBayesDiscrete(data_train)
dp

[          y    0    1    2
 1  0.555556    0  0.2    0
 2       NaN  0.2    0  0.2
 3       NaN  0.2    0    0
 4       NaN  0.2  0.2  0.2
 5       NaN    0    0    0
 6       NaN  0.4  0.6  0.6,           y     0     1     2
 1  0.444444   0.5  0.25  0.25
 2       NaN  0.25   0.5   0.5
 3       NaN     0  0.25     0
 4       NaN     0     0     0
 5       NaN  0.25     0     0
 6       NaN     0     0  0.25]

In [21]:
trainData=pd.concat([y_Label_train,X_Label_train],axis=1)
len(y_Label_train)

9

In [22]:
testdata=pd.concat([y_Label_test,X_Label_test],axis=1)
C=classifyNaiveBayesDiscrete(testdata,dp)
acc=format(100.0*sum(C==y_Label_test)/len(y_Label_test))
print "We correctly classified {0} percents of the trips based on the labeled data only".format(acc)     

We correctly classified 8.33333333333 percents of the trips based on the labeled data only


In [23]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if (sum(np.sum((dp1[0]-dp[0])**2))+sum(np.sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
                
            L=L+math.log(P)
            
        print t,L
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp



In [24]:
#perform EM estimation for theta
dpEM=EM(data_train.iloc[:,1:],data_train.iloc[:,0],data_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(data_test,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==data_test.y)/len(C)
print "After EM we correctly classified {0} percents of the trips".format(acc)

1 -590.452072842
Iteration 1: log maximum liklihood = -590.452072842
2 -572.346722303
Iteration 2: log maximum liklihood = -572.346722303
3 -559.190769042
Iteration 3: log maximum liklihood = -559.190769042
4 -556.633109435
Iteration 4: log maximum liklihood = -556.633109435
5 -556.067798435
Iteration 5: log maximum liklihood = -556.067798435
After EM we correctly classified 94.4444444444 percents of the trips


### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:

#### Question: 

a) Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 

b) Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

In [25]:
data=pd.read_csv("EM_Cluster.csv", index_col = 0)
X=data.iloc[:,1:]
np.random.seed(2016)
dp = trainNaiveBayesDiscrete(X)
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]

for j in range(1,len(dp[0].T)):
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b
dpEM=EM([],[],X,dp)
C = classifyNaiveBayesDiscrete(X,dpEM)

1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan


In [26]:
C

0      1
2      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     0
13     1
14     1
16     1
17     1
18     1
19     1
20     1
21     1
26     0
28     1
29     0
30     1
31     1
32     1
33     1
35     0
37     1
39     1
41     1
44     1
      ..
110    1
112    1
113    1
115    1
116    0
117    0
118    1
121    1
123    1
124    0
125    1
127    1
128    1
129    1
130    1
131    1
133    1
134    1
136    1
137    1
138    1
139    1
140    1
141    1
142    1
144    0
146    0
147    0
148    1
149    1
Name: 1, dtype: int64

In [27]:
np.random.seed(2016)
d = []
C = pd.DataFrame()
#initialize theta randomly
for k in range(10):
    
    dp = trainNaiveBayesDiscrete(X)
    dp[0].iloc[0,0]=np.random.uniform(0,1)
    dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
 
    for j in range(1,len(dp[0].T)):
        b=np.random.uniform(0,1,len(dp[0]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[0].iloc[:,j]=b
    for j in range(1,len(dp[1].T)):
        b=np.random.uniform(0,1,len(dp[1]))
        b=np.asarray(b)/float(np.asarray(b).sum())
        dp[1].iloc[:,j]=b
    dpEM=EM([],[],X,dp) 
    #dpee.append(dpEM)
    c = classifyNaiveBayesDiscrete(X,dpEM)
    d.append(dp)
    C[k] = c

1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iteration 1: log maximum liklihood = nan
2 nan
Iteration 2: log maximum liklihood = nan
3 nan
Iteration 3: log maximum liklihood = nan
1 nan
Iterati

In [28]:
C.mean(axis=1)

0      0.7
2      0.7
4      0.6
5      0.7
6      0.7
7      0.7
8      0.8
9      0.6
10     0.6
11     0.7
13     0.7
14     0.7
16     0.7
17     0.7
18     0.7
19     0.7
20     0.6
21     0.6
26     0.7
28     0.8
29     0.7
30     0.7
31     0.6
32     0.7
33     0.7
35     0.7
37     0.6
39     0.7
41     0.7
44     0.7
      ... 
110    0.6
112    0.6
113    0.8
115    0.7
116    0.7
117    0.7
118    0.8
121    0.6
123    0.7
124    0.7
125    0.6
127    0.6
128    0.7
129    0.6
130    0.7
131    0.7
133    0.7
134    0.8
136    0.8
137    0.6
138    0.7
139    0.6
140    0.8
141    0.7
142    0.8
144    0.7
146    0.7
147    0.7
148    0.7
149    0.6
dtype: float64

In [29]:
C.std(axis = 1)

0      0.483046
2      0.483046
4      0.516398
5      0.483046
6      0.483046
7      0.483046
8      0.421637
9      0.516398
10     0.516398
11     0.483046
13     0.483046
14     0.483046
16     0.483046
17     0.483046
18     0.483046
19     0.483046
20     0.516398
21     0.516398
26     0.483046
28     0.421637
29     0.483046
30     0.483046
31     0.516398
32     0.483046
33     0.483046
35     0.483046
37     0.516398
39     0.483046
41     0.483046
44     0.483046
         ...   
110    0.516398
112    0.516398
113    0.421637
115    0.483046
116    0.483046
117    0.483046
118    0.421637
121    0.516398
123    0.483046
124    0.483046
125    0.516398
127    0.516398
128    0.483046
129    0.516398
130    0.483046
131    0.483046
133    0.483046
134    0.421637
136    0.421637
137    0.516398
138    0.483046
139    0.516398
140    0.421637
141    0.483046
142    0.421637
144    0.483046
146    0.483046
147    0.483046
148    0.483046
149    0.516398
dtype: float64